### Assignment : Week 1 
## Modeling simple RL problems by making their MDPs in Python

We will create the MDPs for some of the example problems from Grokking textbook. For the simple environments, we can just hardcode the MDPs into a dictionary by exhaustively encoding the whole state space and the transition function. We will also go through a more complicated example where the state space is too large to be manually coded and we need to implement the transition function based on some state parameters.

Later on, you will not need to implement the MDPs of common RL problems yourself, most of the work is already done by the OpenAI Gym library, which includes models for most of the famous RL envis.

You can start this assignment during/after reading Grokking Ch-2.

## Environment 0 - Bandit Walk

Let us consider the BW environment on Page 39. 

State Space has 3 elements, states 0, 1 and 2.
States 0 and 2 are terminal states and state 1 is the starting state.

Action space has 2 elements, left and right.

The environment is deterministic - transition probability of any action is 1.

Only 1 (State, Action, State') tuple has positive reward, (1, Right, 2) gives the agent +1 reward.

We'll model this MDP as a dictionary. This code is an example for the upcoming exercises.

In [3]:
bw_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)]
    },

    1 : {
        "Right" : [(1, 2, 1, True)],
        "Left" : [(1, 0, 0, True)]
    },

    2 : {
        "Right" : [(1, 2, 0, True)],
        "Left" : [(1, 2, 0, True)]
    }
    
}

# elements of outer dictionary are various states
# for each state, value is a dict
# for this dict, each key is a possible action from that state
# and each value is a list of a tuple of (transition probability, next state, reward, isTerminal)

Note that by convention, all actions from terminal states still lead to the same state with reward 0.

## Environment 1 - Bandit Slippery Walk

Let us now look at the BSW environment on Page 40. We'll model a slightly modified version of BSW with 7 states instead (on Page 67). It will be useful in the coming weeks.

Here, states 0 and 6 are terminal states and state 3 is the starting state.

Action space has again 2 elements, left and right.

The environment is now stochastic, transition probability of any action is as follows -
If agent chooses `Right` at a non-terminal state,
- $50\%$ times it will go to `Right` state
- $33\frac{1}{3} \%$ times it will stay in same state
- $16\frac{2}{3}\%$ times it will go to `Left`state

This time, 2 different (State, Action, State') tuples have positive rewards, you need to find them.

We'll again model this MDP as a dictionary. Part of the code is written for you.

In [5]:
bsw_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)],
    },

    1 : {
        "Right" : [
            (1/2, 2, 0, False), 
            (1/3, 1, 0, False),
            (1/6, 0, 0, True),
        ],
        "Left" : [
            (1/2, 0, 0, True),
            (1/3, 1, 0, False),
            (1/6, 2, 0, False),
        ]
    }, 
    
    2 : {
        "Right" : [
            (1/2, 3, 0, False), 
            (1/3, 2, 0, False),
            (1/6, 1, 0, False),
        ],
        "Left" : [
            (1/2, 1, 0, False),
            (1/3, 2, 0, False),
            (1/6, 3, 0, False),
        ]
    },
    
    3 : {
        "Right" : [
            (1/2, 4, 1, False), 
            (1/3, 3, 0, False),
            (1/6, 2, 0, False),
        ],
        "Left" : [
            (1/2, 2, 0, False),
            (1/3, 3, 0, False),
            (1/6, 4, 1, False),
        ]
    },
    
    4 : { # reward on this state if reached from other states
        "Right" : [
            (1/2, 5, 0, False), 
            (1/3, 4, 0, False),
            (1/6, 3, 0, False),
        ],
        "Left" : [
            (1/2, 3, 0, False),
            (1/3, 4, 0, False),
            (1/6, 5, 0, False),
        ]
    },
    
    5 : {
        "Right" : [
            (1/2, 6, 0, True), 
            (1/3, 5, 0, False),
            (1/6, 4, 1, False),
        ],
        "Left" : [
            (1/2, 4, 1, False),
            (1/3, 5, 0, False),
            (1/6, 6, 0, True),
        ]
    },
    
    6 : { # essentially same as state 0
        "Right" : [(1, 6, 0, True)],
        "Left" : [(1, 6, 0, True)],
    }
    
}

Feel free to automate filling this MDP, but ensure that it is correctly filled as it'll be back in next week's assignment.

## Environment 2 - Frozen Lake Environment

This environment is described on Page 46.

The FL environment has a large state space, so it's better to generate most of the MDP via Python instead of typing stuff manually.

Note that all 5 states - 5, 7, 11, 12, 15 are terminal states, so keep that in mind while constructing the MDP.

There are 4 actions now - Up, Down, Left, Right.

The environment is stochastic, and states at the border of lake will require separate treatment.



Yet again we will model this MDP as a (large) dictionary.

In [1]:
fl_mdp = {
    # to be added
}

In [23]:
# ACTIONS ARE ASSIGNED NUMBERS AS FOLLOWS: (to match fl env in open ai gym)
# "Left" -> 0
# "Down" -> 1
# "Right" -> 2
# "Up" -> 3

terminal_states = [5, 7, 11, 12, 15]

actions_dict = { # dictionary of actions with their corresponding additions to the current state
    0: -1,
    1: 4,
    2: 1,
    3: -4
}
actions_orthogonal_dict = { # for each key, value is set of actions orthogonal to it
    0: (1, 3),
    1: (0, 2),
    2: (1, 3),
    3: (0, 2),
}

def is_terminal(n):
    return n in terminal_states

def reward(n):
    return float(n == 15) # (to match fl env in open ai gym)

def add_change_to_state(cur_state, addition):
    if abs(addition) == 4: # up or down, check only if new state remains inside lake
        if cur_state + addition in range(16):
            return cur_state + addition
        else:
            return cur_state
            
    elif abs(addition) == 1: # right or left, check only if new state is in same row as original state
        if (cur_state // 4) == ((cur_state + addition) // 4):
            return cur_state + addition
        else:
            return cur_state

for state in range(0, 16):
    # keys of fl_mdp are states
    # values of states are dicts (transitions) with keys actions and values (transition probability, next state, reward, isTerminal)
    transitions = {}

    for action in actions_dict:
        
        if state in terminal_states:
            transitions[action] = [(1.0, state, 0, True)]
            
        else: # current state is not terminal
            action_value = [] # as in, value of action key.
            transition_probability = 1/3 # for all three tuples

            # first tuple: in direction of action
            new_state1 = add_change_to_state(state, actions_dict[action]) # with 1/3 chance
            isTerminal1 = is_terminal(new_state1)
            reward1 = reward(new_state1)

            action_value.append((transition_probability, new_state1, reward1, isTerminal1))

            # second and third tuples: perpendiculat to direction of action
            for orthogonal_action in actions_orthogonal_dict[action]:
                new_statei = add_change_to_state(state, actions_dict[orthogonal_action])
                isTerminali = is_terminal(new_statei)
                rewardi = reward(new_statei)
                action_value.append((transition_probability, new_statei, rewardi, isTerminali))


            transitions[action] = action_value

    fl_mdp[state] = transitions

In [24]:
import pprint
pprint.pprint(fl_mdp) # haan i think now this is equal to the gym ka frozen lake

{0: {0: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)],
     1: [(0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     2: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)],
     3: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)]},
 1: {0: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 1, 0.0, False)],
     1: [(0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 2, 0.0, False)],
     2: [(0.3333333333333333, 2, 0.0, False),
         (0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 1, 0.0, False)],
     3: [(0.3333333333333333,

You might need to do some stuff manually, but make sure to automate most of it.

You can check your implementation of the FL environment by comparing it with the one in OpenAI Gym.

You don't need to worry about Gym right now, we'll set it up in the coming weeks. But here is the code to import an MDP.

In [19]:
import gym
P = gym.make('FrozenLake-v1').env.P

Since the imported MDP is also just a dictionary, we can just print it.

In [20]:
import pprint
pprint.pprint(P)

{0: {0: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False)],
     1: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     2: [(0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)],
     3: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True)],
     1: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 2, 0.0, False)],
     2: [(0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 2, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     3: [(0.3333333333333333,